# RELIEF TOPMRI and Insight46 harmonized datasets
includes data preparation for RELIEF implementations; these are accomplished in an auxilary notebook, the here final data is processed

## import libraries

In [ ]:
import os
import sys

import pandas as pd
import numpy as np

sys.path.insert(0, '../../')  # path to functions
import cvasl.harmony as har

## Import datasets
We will have to flip it on it's side to make it work

In [ ]:
# Datasets for this work
Insight46_path = '../our_datasets/Insight46/'
MRI_path = '../our_datasets/StrokeMRI/'
TOP_path = '../our_datasets/TOP/'
file_name = 'TrainingDataComplete.csv'

TOP_file = os.path.join(TOP_path, file_name)
MRI_file = os.path.join(MRI_path, file_name)
INSI_file = os.path.join(Insight46_path, file_name)


TOP = pd.read_csv(TOP_file, index_col=0)
MRI = pd.read_csv(MRI_file, index_col=0)
INI = pd.read_csv(INSI_file, index_col=0)

In [ ]:
#INI

In [ ]:
sex_mapping = {1:0,2:1}

INI = INI.assign(Sex = INI.Sex.map(sex_mapping))
INI.head(3)

In [ ]:
TOP = TOP.drop(['Site', 'ID'], axis= 1)
TOP.head(3)

In [ ]:
MRI = MRI.drop(['Site', 'ID'],axis = 1)
MRI.tail(3)

In [ ]:
TOPMRI = pd.concat([MRI ,TOP])
TOPMRI.head(3)

In [ ]:
INI = INI.drop(['Site', 'ID'],axis = 1)
INI.tail(3)

In [ ]:
TOPMRI.shape

In [ ]:
both_togetherF, ftF, btF, feature_dictF, len1, len2 = har.prep_for_neurocombat(TOPMRI, INI)

In [ ]:
ftF.to_csv('ftF_top_mri_insiR.csv')

In [ ]:
data = np.genfromtxt('ftF_top_mri_insiR.csv', delimiter=",", skip_header=1)
data = data[:, 1:]
data

In [ ]:
# #save data array for RELIEF processing
np.save('dat_var_for_RELIEF_topmri_v_insight.npy', data)

In [ ]:
TOPMRI.head(3)

In [ ]:
both_togetherF.head(3)

In [ ]:
# Specifying the batch (scanner variable) as well as a sex covariate to preserve:
first_columns_as_one = [1] * len1
last_columns_as_two = [2] * len2
covars = {'batch':first_columns_as_one + last_columns_as_two,
          #'Site': both_togetherF.loc['Site',:].values.tolist(), 
          'Sex':both_togetherF.loc['Sex',:].values.tolist(),
           'Age':both_togetherF.loc['Age',:].values.tolist(),} 
covars = pd.DataFrame(covars) 

In [ ]:
# # save covars for RELIEF processing
covars.to_csv('bath_and_mod_forRELIEF_INISGHT_TOPMRI.csv')


In [ ]:
# #at this point te pre-data is run oin the auxilary notebook to produce "relief1_for5_results.csv"

In [ ]:
topperF = har.make_topper(btF,'Age', 'Sex')

In [ ]:
bottom = pd.read_csv("relief1_for_insight_results.csv", index_col=0)
bottom

In [ ]:
bottom = bottom.reset_index(drop=False)
bottom = bottom.rename(columns={"index": "char"})
bottom.columns = topperF.columns

In [ ]:
back_together = pd.concat([topperF, bottom])
back_together = back_together.T
#back_together

In [ ]:
new_header = back_together.iloc[0] #grab the first row for the header
back_together.columns = new_header #set the header row as the df header
back_together = back_together[1:]
#back_together

In [ ]:
back_together.head(3)

In [ ]:
relief_harm_insight =back_together.tail(len2)
relief_harm_topmri =back_together.head(len1)

In [ ]:
new_feature_dict =  har.increment_keys(feature_dictF)
#new_feature_dict

In [ ]:
relief_harm_topmri_v_i = relief_harm_topmri.rename(new_feature_dict, axis='columns')
relief_harm_insight = relief_harm_insight.rename(new_feature_dict, axis='columns')

In [ ]:
relief_harm_insight = relief_harm_insight.reset_index()
relief_harm_insight = relief_harm_insight.rename(columns={"index": "participant_id"})
relief_harm_insight

Save off to harmonized csv files

In [ ]:
relief_harm_topmri_v_i = relief_harm_topmri_v_i.reset_index()
relief_harm_topmri_v_i = relief_harm_topmri_v_i.rename(columns={"index": "participant_id"})
relief_harm_topmri_v_i 

In [ ]:
#relief_harm_edis.columns

In [ ]:
relief_harm_insight.to_csv('harm_results/relief/relief_harm_insight.csv')
relief_harm_topmri_v_i.to_csv('harm_results/relief/relief_harm_topmri_v_i.csv')

In [ ]:
number_columns = ['Age', 'Sex', 'GM_vol', 'WM_vol', 'CSF_vol',
       'GM_ICVRatio', 'GMWM_ICVRatio', 'WMHvol_WMvol', 'WMH_count',
    'ACA_B_CoV', 'MCA_B_CoV', 'PCA_B_CoV', 'TotalGM_B_CoV',
    'ACA_B_CBF', 'MCA_B_CBF', 'PCA_B_CBF', 'TotalGM_B_CBF',]

# Data quality check

In [ ]:
TOPMRI

In [ ]:
relief_harm_topmri_v_i

In [ ]:
relief_harm_insight.isna().sum().sum()

In [ ]:
relief_harm_topmri.isna().sum().sum()

In [ ]:
#neuro_harm_top.head(3)
(relief_harm_insight[number_columns] < 0).sum()

In [ ]:
(relief_harm_topmri_v_i[number_columns] < 0).sum()

 We will wait about logging it until we see other columns we may want to log across all datasets